<h3 STYLE="background: #c2edff;padding: 0.5em;">Step 4. 線形回帰</h3>

<ol>
<li><a href="#1">「ワインの品質」データ読み込み</a>
<li><a href="#2">単回帰分析</a>
<li><a href="#3">重回帰分析</a>
<li><a href="#4">練習</a>
</ol>
<h4 style="border-bottom: solid 1px black;">Step 4 の目標</h4>

回帰直線を描く。
<img src="fig/linear.png">

In [ ]:
# 数値計算やデータフレーム操作に関するライブラリをインポートする
import numpy as np
import pandas as pd

In [ ]:
# URL によるリソースへのアクセスを提供するライブラリをインポートする。
# import urllib # Python 2 の場合
import urllib.request # Python 3 の場合

In [ ]:
# 図やグラフを図示するためのライブラリをインポートする。
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 線形回帰を行なうライブラリ
from sklearn import linear_model

<h3 STYLE="background: #c2edff;padding: 0.5em;"><a name="1">1. 「ワインの品質」データ読み込み</a></h3>

データは <a href="http://archive.ics.uci.edu/ml/index.php" target="_blank">UC Irvine Machine Learning Repository</a> から取得したものを少し改変しました。

* 赤ワイン https://raw.githubusercontent.com/chemo-wakate/tutorial-6th/master/beginner/data/winequality-red.txt

* 白ワイン https://raw.githubusercontent.com/chemo-wakate/tutorial-6th/master/beginner/data/winequality-white.txt

<h4 style="border-bottom: solid 1px black;">　<a href="http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality.names">詳細</a></h4>

<ol>
<li>fixed acidity : 不揮発酸濃度（ほぼ酒石酸濃度）
<li>volatile acidity : 揮発酸濃度（ほぼ酢酸濃度）
<li>citric acid : クエン酸濃度
<li>residual sugar : 残存糖濃度
<li>chlorides : 塩化物濃度
<li>free sulfur dioxide : 遊離亜硫酸濃度
<li>total sulfur dioxide : 亜硫酸濃度
<li>density : 密度
<li>pH : pH
<li>sulphates : 硫酸塩濃度
<li>alcohol : アルコール度数
<li>quality (score between 0 and 10) : 0-10 の値で示される品質のスコア
</ol>

In [ ]:
# ウェブ上のリソースを指定する
url = 'https://raw.githubusercontent.com/chemo-wakate/tutorial-6th/master/beginner/data/winequality-red.txt'
# 指定したURLからリソースをダウンロードし、名前をつける。
# urllib.urlretrieve(url, 'winequality-red.csv') # Python 2 の場合
urllib.request.urlretrieve(url, 'winequality-red.txt') # Python 3 の場合

In [ ]:
# データの読み込み
df1 = pd.read_csv('winequality-red.txt', sep='\t', index_col=0) 

In [ ]:
df1.head() # 先頭５行まで表示

<h3 STYLE="background: #c2edff;padding: 0.5em;"><a name="2">2. 単回帰分析</a></h3>

まずは、説明変数を１つだけ用いて単回帰分析を行います。

In [ ]:
clf = linear_model.LinearRegression()

In [ ]:
X = df1.loc[:, ['pH']].as_matrix() # 説明変数 = pH

In [ ]:
pd.DataFrame(X).T # 中身の確認。縦に長いと見にくいので転置して表示。

In [ ]:
Y = df1['fixed acidity'].as_matrix() # 目的変数 = fixed acidity

In [ ]:
pd.DataFrame(Y).T # 中身の確認。縦に長いと見にくいので転置して表示。

In [ ]:
clf.fit(X, Y) # 予測モデルを作成

In [ ]:
# 回帰係数
clf.coef_

In [ ]:
# 切片
clf.intercept_

以上の結果は、説明変数と目的変数の関係が次の回帰式で表されることを示しています。

決定係数は寄与率とも呼ばれ、説明変数が目的変数のどれくらいを説明できるかを表す値であり、１に近いほどモデルの精度が高いことを表します。

In [ ]:
# 決定係数
clf.score(X, Y)

結果を 2 次元座標上にプロットすると、以下のようになります。青線が回帰直線を表します。

In [ ]:
# 散布図
plt.scatter(X, Y)
 
# 回帰直線
plt.title('Linear regression')
plt.plot(X, clf.predict(X))
plt.xlabel('pH')
plt.ylabel('fixed acidity')
plt.grid()
plt.show()

<h3 STYLE="background: #c2edff;padding: 0.5em;"><a name="3">3. 重回帰</a></h3>

続いて、「quality」を目的変数に、「quality」以外を説明変数として、重回帰分析を行います。

In [ ]:
clf = linear_model.LinearRegression()
 
# 説明変数に "quality (品質スコア以外すべて)" を利用
df1_except_quality = df1.drop('quality', axis=1)
X = df1_except_quality.as_matrix()
 
# 目的変数に "quality (品質スコア)" を利用
Y = df1['quality'].as_matrix()
 
# 予測モデルを作成
clf.fit(X, Y)
  
# 切片 (誤差)
print(clf.intercept_)

# 偏回帰係数
pd.DataFrame({'Name':df1_except_quality.columns, 'Coefficients':clf.coef_}).sort_values('Coefficients', ascending=False) 

上記の結果からワインの品質スコアは、以下のような回帰式で表せることがわかります。

ただし決定係数は次の値です。

In [ ]:
clf.score(X, Y)

各変数がどの程度目的変数に影響しているかを確認するには、各変数を正規化 (標準化) し、平均 = 0, 標準偏差 = 1 になるように変換した上で、重回帰分析を行うと偏回帰係数の大小で比較することができるようになります。

In [ ]:
clf = linear_model.LinearRegression()
 
# データフレームの各列を正規化
df1s = df1.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
 
# 説明変数に "quality (品質スコア以外すべて)" を利用
df1s_except_quality = df1s.drop("quality", axis=1)
X = df1s_except_quality.as_matrix()
 
# 目的変数に "quality (品質スコア)" を利用
Y = df1s['quality'].as_matrix()
 
# 予測モデルを作成
clf.fit(X, Y)
 
# 偏回帰係数
pd.DataFrame({'Name':df1s_except_quality.columns,
                    'Coefficients':np.abs(clf.coef_)}).sort_values('Coefficients', ascending=False) 

正規化した偏回帰係数を確認すると、alcohol (アルコール度数) が最も高い値を示し、品質に大きな影響を与えていることがわかります。

<h4 style="padding: 0.25em 0.5em;color: #494949;background: transparent;border-left: solid 5px #7db4e6;"><a name="4">練習4.1</a></h4>

白ワインのデータ(https://raw.githubusercontent.com/chemo-wakate/tutorial-6th/master/beginner/data/winequality-white.txt) を読み込み、単回帰分析、重回帰分析を行なってください。

In [ ]:
# 練習4.1